In [21]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy

from package.helpers.test_physical_lens import TestPhysicalLens
from package.helpers.lens import Lens
from package.plots import diagnostic_plot, plot
from package.utils import util
from package.scripts import generate

plt.style.use(f'{repo_path}/package/mplstyle/science.mplstyle')

In [23]:
data_dir = os.path.join('/data','bwedig', 'roman-pandeia')
array_dir = os.path.join(data_dir, 'output', 'arrays', 'multiprocessing')
util.create_directory_if_not_exists(array_dir)

# get path to Roman instrument params CSV
roman_params_csv = os.path.join(repo_path, 'data', 'roman_spacecraft_and_instrument_parameters.csv')

# get output of SkyPy pipeline
df = pd.read_csv(os.path.join('/data','bwedig', 'roman-population', 'data', 'dictparaggln_Area00000010.csv'))

In [24]:
limit = 2
total = deepcopy(limit)
lens_list = []

In [25]:
# generate the lens objects
for i, row in tqdm(df.iterrows(), total=total):
    if i == limit:
        break

    # select only the cool ones lmao
    # if row['numbimag'] == 1.0:
    #     limit += 1
    #     continue

    lens = Lens(z_lens = row['redslens'], 
                z_source=row['redssour'], 
                sigma_v=row['velodisp'], 
                lens_x=row['xposlens'], 
                lens_y=row['yposlens'], 
                source_x=row['xpossour'], 
                source_y=row['ypossour'], 
                mag_lens=row['magtlensF106'], 
                mag_source=row['magtsourF106'])
    
    lens_list.append(list)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 57.71it/s]


In [26]:
# TODO testing for now
lens_list = [TestPhysicalLens] * limit

In [27]:
core_count = multiprocessing.cpu_count()

In [28]:
from multiprocessing import Pool

pool = Pool(processes=2)  # TODO core_count
output = pool.map(generate.main, lens_list)

In [20]:
process_list = []

for i in range(limit):
    process_list.append(multiprocessing.Process(target=generate(lens, roman_params_csv)))

TypeError: 'module' object is not callable

In [ ]:
np.save(os.path.join(array_dir, f'skypy_output_{str(i).zfill(5)}'), image)

In [ ]:
# save lists of execution times
np.save(os.path.join(array_dir, 'skypy_output_lens_execution_times.npy'), lens_execution_times)
np.save(os.path.join(array_dir, 'skypy_output_pandeia_execution_times.npy'), pandeia_execution_times)
np.save(os.path.join(array_dir, 'skypy_output_estimated_times.npy'), estimated_times)